In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
!pip install kaggle --quiet
import json
import os

In [ ]:
from google.colab import userdata
#Load kaggle credentials
kaggle_credentials = {"username":userdata.get('username'),
                      "key":userdata.get('key')}

In [ ]:
# Create the .kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Save the credentials to kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle_credentials, f)

# Set the permissions for the kaggle.json file
os.chmod('/root/.kaggle/kaggle.json', 0o600)

In [ ]:
!kaggle datasets download -d rmisra/news-category-dataset

Dataset URL: https://www.kaggle.com/datasets/rmisra/news-category-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 19% 5.00M/26.5M [00:00<00:00, 32.6MB/s]
100% 26.5M/26.5M [00:00<00:00, 111MB/s] 


In [ ]:
# Unzip the downloaded file
import zipfile

with zipfile.ZipFile('news-category-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()


In [ ]:
#Load the dataset into a pandas DataFrame
df = pd.read_json('News_Category_Dataset_v3.json', lines =True)
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [ ]:
categories = df['category'].unique()